In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import datetime
import shutil

In [68]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample = pd.read_csv('./data/sample_submission.csv')

In [69]:
train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


In [70]:
test

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [71]:
sample

,ID,y
0,1,100.669318
1,2,100.669318
2,3,100.669318
3,4,100.669318
4,5,100.669318
...,...,...
4204,8410,100.669318
4205,8411,100.669318
4206,8413,100.669318
4207,8414,100.669318


In [72]:
train.nunique()[train.nunique()>20]

ID    4209
y     2545
X0      47
X1      27
X2      44
X5      29
X8      25
dtype: int64

In [73]:
int_type_idx = train.dtypes[train.dtypes == ('int64')].index
int_type_idx.values

array(['ID', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17',
       'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X26', 'X27',
       'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36',
       'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45',
       'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54',
       'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63',
       'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X73',
       'X74', 'X75', 'X76', 'X77', 'X78', 'X79', 'X80', 'X81', 'X82',
       'X83', 'X84', 'X85', 'X86', 'X87', 'X88', 'X89', 'X90', 'X91',
       'X92', 'X93', 'X94', 'X95', 'X96', 'X97', 'X98', 'X99', 'X100',
       'X101', 'X102', 'X103', 'X104', 'X105', 'X106', 'X107', 'X108',
       'X109', 'X110', 'X111', 'X112', 'X113', 'X114', 'X115', 'X116',
       'X117', 'X118', 'X119', 'X120', 'X122', 'X123', 'X124', 'X125',
       'X126', 'X127', 'X128', 'X129', 'X130', 'X131', 'X132', 'X133',
       'X134', '

In [74]:
train.loc[:,np.append(int_type_idx.values,'y')].corr()['y'].sort_values(ascending=False).head(10)

y       1.000000
X314    0.606005
X261    0.588785
X263    0.379629
X136    0.368177
X275    0.275042
X118    0.272728
X119    0.272728
X51     0.229999
X264    0.228169
Name: y, dtype: float64

モデルの構築

In [75]:
X = train[['X314','X261','X263']]
y = train['y']
X_test = test[['X314','X261','X263']]

In [76]:
kf = KFold(n_splits=5)

In [77]:
model_list = []

for train_idx,valid_idx in kf.split(X,y):
    X_train,X_valid = X.iloc[train_idx,:],X.iloc[valid_idx,:]
    y_train,y_valid = y[train_idx],y[valid_idx]
    model = LinearRegression()
    model.fit(X_train,y_train)
    model_list.append(model)

In [78]:
train_r2_score_list = []
valid_r2_score_list = []

for model in model_list:
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    train_r2_score_list.append(r2_score(y_train,y_train_pred))
    valid_r2_score_list.append(r2_score(y_valid,y_valid_pred))

print('train_r2_score_list:',train_r2_score_list)
print('valid_r2_score_list:',valid_r2_score_list)

train_r2_score_list: [0.42641336049814127, 0.42550322806740704, 0.4265393176332253, 0.4267777181686099, 0.42845628981730854]
valid_r2_score_list: [0.5038105485174784, 0.5069893494852091, 0.503326716511209, 0.5023556239964773, 0.4852826419218971]


テストデータの予測

In [83]:
y_test_df = pd.DataFrame()

for i,model in enumerate(model_list):
    y_test = model.predict(X_test)
    y_test = pd.Series(y_test,name=f'model_{i}')
    y_test_df = pd.concat([y_test_df,y_test],axis=1)

y_test_df['mean'] = y_test_df.apply(np.mean,axis=1)
y_test_df

,model_0,model_1,model_2,model_3,model_4,mean
0,78.094430,77.699650,77.659928,78.104527,78.243034,77.960314
1,95.228836,95.085506,95.176604,95.196606,95.727678,95.283046
2,78.094430,77.699650,77.659928,78.104527,78.243034,77.960314
3,78.094430,77.699650,77.659928,78.104527,78.243034,77.960314
4,109.272236,109.199544,109.391704,109.420923,109.958000,109.448481
...,...,...,...,...,...,...
4204,109.272236,109.199544,109.391704,109.420923,109.958000,109.448481
4205,95.228836,95.085506,95.176604,95.196606,95.727678,95.283046
4206,95.228836,95.085506,95.176604,95.196606,95.727678,95.283046
4207,109.272236,109.199544,109.391704,109.420923,109.958000,109.448481


In [123]:
submit = pd.concat([test['ID'],y_test_df['mean']],axis=1)
now = datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
submit_filename = f'submit_{now}' + '.csv'
submit.to_csv(submit_filename)
shutil.move(submit_filename,'./submit_files')

'./submit_files/submit_2022-07-08-10:32:07.csv'